<a href="https://colab.research.google.com/github/abhijithneilabraham/ExploreML/blob/master/agedetection10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Parasgr7/Age-Detection-Indian-Actors-

Cloning into 'Age-Detection-Indian-Actors-'...
remote: Enumerating objects: 26096, done.
remote: Total 26096 (delta 0), reused 0 (delta 0), pack-reused 26096
Receiving objects: 100% (26096/26096), 57.17 MiB | 27.19 MiB/s, done.
Resolving deltas: 100% (7405/7405), done.
Checking out files: 100% (26550/26550), done.


In [2]:
%cd Age-Detection-Indian-Actors-


/content/Age-Detection-Indian-Actors-


Using TensorFlow backend.


In [0]:
import os
import random
import numpy as np
import keras
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from cv2 import imread,resize
from keras.optimizers import SGD, Adam, Adamax
import cv2
import tensorflow as tf

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [0]:
temp = []
for img_name in train.ID:
    img_path ='Train/'+img_name
    img = imread(img_path)
    img = resize(img, (64,64))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img=cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    img = img.astype('float32') # this will help us in later stage
    temp.append(img)

train_x = np.stack(temp)



In [0]:
temp = []
for img_name in test.ID:
    img_path = 'Test/'+img_name
    img = imread(img_path)
    img = resize(img, (64,64))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img=cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    img = img.astype('float32') # this will help us in later stage
    temp.append(img)
test_x = np.stack(temp)

In [0]:


train_x = train_x / 255.
test_x = test_x / 255.
train.Class.value_counts(normalize=True)

lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.np_utils.to_categorical(train_y)

In [54]:
epochs = 15
batch_size = 128

model = Sequential()
model.add(Convolution2D(32, (3, 3),  padding='valid',activation='relu', input_shape=(64,64,3)))

model.add(Convolution2D(16, (3, 3), padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(8, (3, 3), padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))




model.summary()
model.save_weights('weights.h5')
model.load_weights('weights.h5')
model.compile(optimizer=Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-9, amsgrad=False), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=batch_size,epochs=epochs,verbose=1,validation_split=0.1)


pred = model.predict_classes(test_x)
pred = lb.inverse_transform(pred)

test['Class'] = pred
test.to_csv('sub.csv', index=False)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_41 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 60, 60, 16)        4624      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 28, 28, 8)         1160      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 14, 14, 8)         0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 1568)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)             